# LangGraph with Multi-Agent Workflow 2 - Own MOD

Multiple agents interacting in Workflow design


In [2]:
import os
import json
import operator
import functools
from loguru import logger
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage
)
from langchain.tools import tool
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import FunctionMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_experimental.utilities import PythonREPL
from typing import TypedDict, Annotated, List, Sequence, Literal
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END

In [3]:
# OpenAI API Key solely to use embedding model
os.environ["OPENAI_API_KEY"] = ""
os.environ["TAVILY_API_KEY"] = ""

In [7]:
# Load LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

In [8]:
sample_query = """
I want to estimate the impact of a drug dosage on an overall survival
I don't want to incur in immortality bias.
What would the rigth epidemeological approach to do this?
"""
sample_result = llm.invoke(sample_query)

In [10]:
sample_result.pretty_print()

================================== Ai Message ==================================

To estimate the impact of a drug dosage on overall survival while avoiding immortality bias, you should consider using time-dependent methods in your epidemiological analysis. Immortality bias occurs when there is a period during which the outcome (death, in this case) cannot occur, often because the exposure (drug dosage) has not yet been initiated. Here are some approaches to mitigate this bias:

1. **Time-Dependent Cox Proportional Hazards Model**: This model allows you to account for changes in drug dosage over time. By treating drug dosage as a time-dependent covariate, you can more accurately estimate its impact on survival.

2. **Landmark Analysis**: This involves selecting a specific time point (landmark) and including only those patients who have survived up to that point. You then analyze the impact of drug dosage on survival from the landmark time onward. This helps to ensure that all patients 

In [11]:
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

### Define Tools

In [ ]:
tavily_tool = TavilySearchResults(max_results=5)

repl = PythonREPL()

@tool
def python_repl(code: Annotated[str, "The python code to execute to generate your chart."]):
    
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""

    try:
        result = repl.run(code)
        
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )